In [ ]:
###############################################################################################
###############################################################################################
# Part 1: work with shapefiles
# I am using a "shapefile" which consists of at least four actual files (.shp, .shx, .dbf, .prj). This is a commonly used format.
# The new ".rds" format shapefiles seem to be designed only for use in R programming (For more about shapefile formats: https://gadm.org/formats.html).
# An example shapefiles source: https://gadm.org/download_country_v3.html
###############################################################################################
# Method 1 (Matplotlib + Cartopy)
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature

# set up working directory
import os
os.chdir("C:\Study\Shapefile\gadm36_GBR_shp")

# import the shapefile
UK_shape_file = r'gadm36_GBR_3.shp'
# get the map (geometries)
UK_map = ShapelyFeature(Reader(UK_shape_file).geometries(),ccrs.PlateCarree(), edgecolor='black',facecolor='none')
# initialize a plot
test= plt.axes(projection=ccrs.PlateCarree())
# add the shapefile for the whole UK
test.add_feature(UK_map) 
# zoom in to London
test.set_extent([-2,2,51,52], crs=ccrs.PlateCarree()) 
###############################################################################################
# Method 2 (Matplotlib + Cartopy + Geopandas)
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd

# set up working directory
import os
os.chdir("C:\Study\Shapefile\gadm36_GBR_shp")

# read the UK shapefile as a "geopandas.geodataframe.GeoDataFrame"
UK_shapefile = gpd.read_file("gadm36_GBR_3.shp")
# check what UK shapefile contains
UK_shapefile
# get shapes for London, Birmingham and Edinburgh from the UK shapefile
# you can also go to a coarser/finer layer to select a bigger/smaller domain
London = UK_shapefile[UK_shapefile['NAME_2'] == "Greater London"]
Birmingham = UK_shapefile[UK_shapefile['NAME_2'] == "Birmingham"]
Edinburgh  = UK_shapefile[UK_shapefile['NAME_2'] == "Edinburgh"]

# check the geometry for each city
print(London.geometry)
print(Birmingham.geometry)
print(Edinburgh.geometry)

# create a list of your study cities and merge the shapes (geopandas.geodataframe.GeoDataFrame)
import pandas as pd
study_cities = [London,Birmingham,Edinburgh]
study_cities_shapes = gpd.GeoDataFrame(pd.concat(study_cities, ignore_index=True))

# initialize a plot
test= plt.axes(projection=ccrs.PlateCarree())
# add shapefiles to your chosen cities only
# you can change "edgecolor", "facecolor" and "linewidth" to highlight certain areas
# you can change the "zorder" to decide the layer
test.add_geometries(study_cities_shapes.geometry, crs=ccrs.PlateCarree(),edgecolor='black',facecolor='none',linewidth=2,zorder=0)

# zoom in to your study domain
test.set_extent([-5,2,51,57], crs=ccrs.PlateCarree())

# Part 1 Remarks:
# 1> I prefer Method 2 as "geopandas" allows more control of the shapefile
# 2> But it is almost impossible to install geopandas following the instructions on its homepage: https://geopandas.org/install.html
#    I managed to install it on my windows PC following this video: https://www.youtube.com/watch?v=LNPETGKAe0c
# 3> Method 1 is easy to use on all platforms, althought there is less control of the shapefile
###############################################################################################
# Part 2: some techniques with "polygons"
# sometimes, we may want to know which data sites are within our outside a certain area
# or we may want to know if two areas have any overlaps
# use can use some tricks with "polygons" to achieve these
###############################################################################################
# task 1: create a polygon using shapefiles (using Chinese cities as an example)

# read shapefiles for cities in mainland China
os.chdir("/rds/projects/2018/maraisea-glu-01/Study/Research_Data/BTH/domain/gadm36_CHN_shp")
China_cities = gpd.read_file("gadm36_CHN_2.shp")

# check the city list
print(China_cities['NAME_2'])

# get the list of your target cities
os.chdir("/rds/projects/2018/maraisea-glu-01/Study/Research_Data/BTH/domain/")
BTH_cities = pd.read_csv("2+26_cities.csv")
BTH_cities = list(BTH_cities['City'])

# extract the shape (multi-polygon) for each city
BTH_shapes = [China_cities[China_cities['NAME_2'] == city_name] for city_name in BTH_cities]
print("Number of city shapefiles:",len(BTH_shapes))

# combine shapefiles from all cities into a single shape
BTH_shapes = gpd.GeoDataFrame(pd.concat(BTH_shapes, ignore_index=True))

# check the shape for a certain city
BTH_shapes['geometry'][0]

# plot the combined shape for the target cities
from shapely.ops import cascaded_union
BTH_polygons = BTH_shapes['geometry']
BTH_boundary = gpd.GeoSeries(cascaded_union(BTH_polygons))
BTH_boundary.plot(color = 'red')
plt.show()
###############################################################################################
# task 2: derive the polygon for a grid centre with given resolutions (use GEOS-Chem model grids as the example)
def create_polygon(lon,lat,lon_res,lat_res):
  '''Input lon,lat,resolution of lon,lat in order. Then create the polygon for the target grid'''
    from shapely import geometry
    from shapely.ops import cascaded_union
    p1 = geometry.Point(lon-lon_res/2,lat-lat_res/2)
    p2 = geometry.Point(lon+lon_res/2,lat-lat_res/2)
    p3 = geometry.Point(lon+lon_res/2,lat+lat_res/2)
    p4 = geometry.Point(lon-lon_res/2,lat+lat_res/2)
    pointList = [p1, p2, p3, p4, p1]
    output_polygon = geometry.Polygon([[p.x, p.y] for p in pointList])
    output_polygon = gpd.GeoSeries(cascaded_union(poly))
    return output_polygon
  
# based on this, you can also create your function to return a polygon using coordiantes of of data points.  
 ###############################################################################################
# task 3: test if a polygon contains a certain point
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

point = Point(0.5, 0.5)
polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
print(polygon.contains(point))
###############################################################################################
# task 4: test if two polygons have any overlaps
print(polyon_A.intersects(polyon_B))

# Part 2 Remarks:
# these can be useful as sometimes we want to know which grid contain our target data points 
# or we may want to know if which grids are within the target domain
# or we may want to know some details of data in a certain domain

# End
###############################################################################################
